In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import numpy as np
#import jplus



q chucha


Using TensorFlow backend.
/Users/aaorsi/anaconda/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,


In [2]:
#load databases

#DataDir = './data/'
DataDir = '.'
import pickle
dset = pickle.load(open('%s/dataset.data'%DataDir))
print dset.keys()

print 

x_train = np.asarray(dset['tfeatures'])
y_train = np.asarray(dset['tclass'])
x_test = np.asarray(dset['vfeatures'])
y_test    = np.asarray(dset['vclass'])


nfeat = len(x_train[0])
print nfeat
print np.unique(y_train)
print 'n_dataset:%ld'%len(y_train)
#blippi

['tfeatures', 'vclass', 'tclass', 'vfeatures']

19
['Halpha' 'OII' 'OIII+Hbeta' 'contaminant']
n_dataset:95300


In [6]:
#Example sets

kx_train = np.random.random((1000, 20))
ky_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
kx_test = np.random.random((100, 20))
ky_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

print np.shape(kx_train), np.shape(x_train)
print np.shape(ky_train), np.shape(y_train)

def class_to_int(istr):
    if istr == 'Halpha':
        return 0
    elif istr == 'OIII+Hbeta':
        return 1
    if istr == 'OII':
        return 2
    elif istr == 'contaminant':
        return 3
    else:
        print '%s not recognised'%istr
        return -99


y_train_int = [class_to_int(x) for x in y_train]
y_test_int = [class_to_int(x) for x in y_test]

y2_train = keras.utils.to_categorical(y_train_int, num_classes=len(np.unique(y_train_int)))
y2_test = keras.utils.to_categorical(y_test_int, num_classes=len(np.unique(y_test_int)))



(1000, 20) (95300, 19)
(1000, 10) (95300,)


Scale data using scikit-learn StandardScaler

In [7]:
from sklearn.preprocessing import StandardScaler 
Scaledata = True

if Scaledata:
    print 'scaling data...',
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test  = scaler.transform(x_test)
    print 'done'


scaling data... done


In [10]:
# Keras MLP

from keras import metrics

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(19, activation='relu', input_dim=nfeat))
#model.add(Dropout(0.5))
model.add(Dense(19, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=[metrics.categorical_accuracy])
print 'doing stuff'

print np.shape(x_train)
print np.shape(y2_train)

model.summary()



doing stuff
(95300, 19)
(95300, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 19)                380       
_________________________________________________________________
dense_5 (Dense)              (None, 19)                380       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 80        
Total params: 840
Trainable params: 840
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train, y2_train,
          epochs=20,
          batch_size=500,
          validation_data=(x_test,y2_test),
          shuffle=True,
          verbose=2
          )



Train on 95300 samples, validate on 95300 samples
Epoch 1/20
 - 1s - loss: 0.4535 - categorical_accuracy: 0.8114 - val_loss: 0.4489 - val_categorical_accuracy: 0.8104
Epoch 2/20
 - 1s - loss: 0.4477 - categorical_accuracy: 0.8136 - val_loss: 0.4464 - val_categorical_accuracy: 0.8212
Epoch 3/20
 - 1s - loss: 0.4424 - categorical_accuracy: 0.8159 - val_loss: 0.4394 - val_categorical_accuracy: 0.8193
Epoch 4/20
 - 1s - loss: 0.4378 - categorical_accuracy: 0.8181 - val_loss: 0.4336 - val_categorical_accuracy: 0.8206
Epoch 5/20
 - 1s - loss: 0.4332 - categorical_accuracy: 0.8201 - val_loss: 0.4314 - val_categorical_accuracy: 0.8259
Epoch 6/20
 - 1s - loss: 0.4289 - categorical_accuracy: 0.8222 - val_loss: 0.4259 - val_categorical_accuracy: 0.8303
Epoch 7/20
 - 2s - loss: 0.4252 - categorical_accuracy: 0.8245 - val_loss: 0.4216 - val_categorical_accuracy: 0.8250
Epoch 8/20
 - 2s - loss: 0.4216 - categorical_accuracy: 0.8262 - val_loss: 0.4177 - val_categorical_accuracy: 0.8214
Epoch 9/20
 - 

In [24]:
pred = model.predict(x_train,verbose=1)
pred_class = [np.argmax(x) for x in pred]
#print x_train[0:10]


95300/95300 [==============================] - 3s 28us/step


In [25]:
from sklearn.metrics import f1_score
score = f1_score(y_train_int, pred_class,average=None)
print score
print np.unique(pred_class)

[0.95306203 0.796263   0.83577079 0.78684193]
[0 1 2 3]
